# Geometry
## Fiona and Shapely Test

Feature geometry can be accessed using the `geometry` property of each feature, for example:

In [5]:
import fiona

with fiona.open("../data/europe_110.geo.gpkg") as eu:
    print(eu[10]["geometry"])
    

{'type': 'MultiPolygon', 'coordinates': [[[(12.690006137755631, 55.609990953180784), (12.089991082414741, 54.80001455343793), (11.043543328504228, 55.364863796604254), (10.903913608451631, 55.77995473898875), (12.370904168353292, 56.111407375708836), (12.690006137755631, 55.609990953180784)]], [[(10.912181837618363, 56.458621324277914), (10.667803989309988, 56.08138336854722), (10.369992710011985, 56.19000722922473), (9.649984978889307, 55.469999498102055), (9.921906365609175, 54.98310415304806), (9.282048780971138, 54.83086538351617), (8.526229282270236, 54.96274363872499), (8.12031090661759, 55.517722683323626), (8.08997684086225, 56.5400117051376), (8.256581658571264, 56.8099693874303), (8.543437534223386, 57.110002753316905), (9.42446902836761, 57.17206614849948), (9.775558709358563, 57.44794078228966), (10.580005730846153, 57.73001658795485), (10.546105991262692, 57.215732733786155), (10.250000034230226, 56.89001618105047), (10.369992710011985, 56.609981594460834), (10.91218183761

### Using Shapely for geometry manipulation   

[Shapely](http://toblerity.org/shapely/manual.html) is a Python package for 
set-theoretic analysis and manipulation of planar features using (via Python’s ctypes module) 
functions from the well-known and widely deployed GEOS library. 
GEOS, a port of the Java Topology Suite (JTS), is the geometry engine of the PostGIS spatial 
extension for the PostgreSQL RDBMS. 
The designs of JTS and GEOS are largely guided by the 
Open Geospatial Consortium‘s Simple Features Access Specification and 
Shapely adheres mainly to the same set of standard classes and operations. 
Shapely is thereby deeply rooted in the conventions of the geographic information systems (GIS) world, 
but aspires to be equally useful to programmers working on non-conventional problems.

With Shapely, we can solve tasks like

* How big is the feature
* Do two features overlap?
* How does the common area of two features look like
* Create a buffer area around the feature
* ...

### Converting JSON to geometry objects

First we import `shapely` and it's functions and then convert the JSON-encoded geometries to Geometry objects

In [ ]:
from shapely.geometry import shape

with fiona.open("../data/europe_110.geo.json") as eu:
    geom = shape(eu[14]["geometry"])
geom # Jupyter can display geometry data directly

In [ ]:
print(geom.type)

In [ ]:
print(geom.area)

In [ ]:
print(geom.length)

Let's have a look at some geometry methods

In [ ]:
help(geom)

For example we can make a buffer around our polygon:

In [ ]:
geom.buffer(500)

## Converting the geometry back to JSON format
Once we are finished, we can convert the geometry back to JSON format using `shapely.geometry.mapping` function


In [ ]:
from shapely.geometry import mapping

In [ ]:
# let's create new GeoJSON-encoded vector feature
import copy

new_feature = {
    "type": "Feature",
    "properties": {"name": "My buffered feature"},
    "geometry": mapping(geom.buffer(100))
}
new_feature

---
[<- Introduction](01-introduction.ipynb) | [Projections ->](03-projections.ipynb)